# A simple demo for the OpenHowNet Python Package

To begin with, make sure you have installed **Python 3.X**. The **anytree** dependency is required to be installed.

It is our only required dependency because other python packages we need in building the OpenHowNet Python Package will be defaultly installed with the Python 3.X.

Then you should download the **OpenHowNet Package**. Please checkout the installation by import the module like the following code:

In [ ]:
# Import the OpenHowNet module
import OpenHowNet

After that we can build a **HowNetDict**:

In [ ]:
# Initialize the HowNetDict, you can initialize the similarity calculation module by setting the init_sim to True.
hownet_dict = OpenHowNet.HowNetDict(init_sim=False)

Now the preparation work is all done. Then let's explore some important features of HowNetDict.

## Basic Usage of OpenHowNet Python Package

### Get word annotations in HowNet

By default, the api will search the target word in both English and Chinese annotations in HowNet, which will cause significant search overhead. Note that if the target word does not exist in HowNet annotation, this api will simply return an empty list.

In [ ]:
# Get the senses list annotated with "苹果".
result_list = hownet_dict.get_sense("苹果")
print("The number of retrievals: ", len(result_list))
print("An example of retrievals: ", result_list)

In [ ]:
# Get the detailed information of the sense.
sense_example = result_list[0]
print("Sense example:", sense_example)
print("Sense id: ",sense_example.No)
print("English word in the sense: ", sense_example.en_word)
print("Chinese word in the sense: ", sense_example.zh_word)
print("HowNet annotation of the sense: ", sense_example.Def)
print("Sememe list of the sense: ", sense_example.get_sememe_list())

In [ ]:
# Get the detailed information of the sememe.
sememe_example = sense_example.get_sememe_list().pop()
print("Sememe example: ", sememe_example)
print("The English annotation of the sememe: ", sememe_example.en)
print("The Chinese annotation of the sememe: ", sememe_example.zh)
print("The frequency of occurrence of the sememe in HowNet: ", sememe_example.freq)

You can visualize the retrieved HowNet structured annotations ("sememe tree") of sense as follow :

In [ ]:
sense_example.visualize_sememe_tree()

To boost the efficiency of the search process, you can specify the language of the target word as the following.

In [ ]:
print("The number of mixed search results:",len(hownet_dict.get_sense("X")))
print("The number of Chinese results:",len(hownet_dict.get_sense("X",language="zh")))
print("The number of English results:",len(hownet_dict.get_sense("X",language="en")))

You can limit the POS of the target word by setting the `pos`.  Besides, you can set the `strict` to false to make a fuzzy match.

In [ ]:

res = hownet_dict.get_sense("苹果", strict=False)
print("Fuzzy match: (The number of retrievals: {})".format(len(res)))
print("Retrivals: {}\n".format(res))
res = hownet_dict.get_sense("苹果",pos='adj', strict=False)
print("Fuzzy match and limit the POS to adj: (The number of retrievals: {})".format(len(res)))
print("Retrivals: {}".format(res))


You can get all senses by setting the `word` to *. Besides, setting the `word` to the sense id also works.

In [ ]:
all_senses = hownet_dict.get_sense('*')
print("The number of all senses: {}".format(len(all_senses)))

Besides, you can also get all the English or Chinese words in HowNet annotations.

In [ ]:
zh_word_list = hownet_dict.get_zh_words()
en_word_list = hownet_dict.get_en_words()
print("Chinese words in HowNet: ",zh_word_list[:30])
print("English words in HowNet: ",en_word_list[:30])

### Get Sememe Trees for certain word in HowNet¶

You can get the sememes by certain word in a variety of forms of presentation. Detailed explanation of params will be displayed in our documentation.
First, you can retrieve all the senses annotated with the certain word and their sememes.

In [ ]:
# Get the respective sememe list of the senses annotated with the word.
# The word can be English or Chinese or *
hownet_dict.get_sememes_by_word(word = '苹果', display='list', merge=False, expanded_layer=-1, K=None)

The `display` can be set to "tree"/"dict"/"list"/"visual", and the function will return in different forms.
1. When set to "list", the sememes will be returned in the form of list as shown above.
2. When set to "dict", the function will return the sememe tree in the form of dict as shown below.

In [ ]:
hownet_dict.get_sememes_by_word(word = '苹果', display='dict', merge=False, expanded_layer=-1, K=None)[0]

3. When set to "tree", the function will return the senses and the root node of their respective sememe tree. 

In [ ]:
t = hownet_dict.get_sememes_by_word(word = '苹果', display='tree', merge=False, expanded_layer=-1, K=None)[0]
print(t)
print("The type of the root node is:", type(t['sememes']))

4. When set to "visual", the function will visualize the topK sememe trees. At this point, `K` can be set to control the num of the visualized sememe trees. 

In [ ]:
hownet_dict.get_sememes_by_word(word = '苹果', display='visual', merge=False, expanded_layer=-1, K=2)

5. `merge` and `expanded_layer` only work when `display=="list"`. When `merge==True`, the sememe lists of all the senses retrieved by the word will be merged into one. `expanded_layer` is set to control the expanded layer num of the sememe tree and by default it will be set to -1(expanded all layers).

In [ ]:
# Expand all layers and merge all the sememe list into one
hownet_dict.get_sememes_by_word(word = '苹果', display='list', merge=True, expanded_layer=-1, K=None)

In [ ]:
# Expand the top2 layers and merge all the sememe list into one. Note that the first layer is the sense node. 
hownet_dict.get_sememes_by_word(word = '苹果', display='list', merge=True, expanded_layer=2, K=None)

### Get sememes via relations between sememes

There are various relations between sememes. The package provides api to retrieve related sememes.